In [1]:
import pandas as pd
from neo4j.exceptions import ClientError, CypherSyntaxError
from src.llm_query_generator.llm import OpenAILLM
from src.llm_query_generator.pipelines import DataBaseDescriptor, AgentPipeline, QAPipeline
from src.llm_query_generator.pipelines.qa_pipeline import clean_generation,format_result_for_qa
from src.llm_query_generator.chat_history import ChatHistory
from src.llm_query_generator.db import Neo4jAdapter
import json
import os
from tqdm import tqdm
from time import time

# Database Selection Evaluation

In [3]:
DATA_BASE_DESCRIPTORS = [
    DataBaseDescriptor(
        "MovieDatabase", "A neo4j database containing information about movies, actors and directors", None
    ),
        DataBaseDescriptor(
        "CLEVR", "A neo4j database containing information about underground train stations and underground train lines with their architecture and other addtitional informations", None
    ),
    DataBaseDescriptor(
        "Northwind", "A neo4j database containing an traditional retail-system with products, orders, customers, suppliers and employees", None
    )
]

JSON_LLM = OpenAILLM(
    api_key=os.environ.get("OPENAI_KEY"),
    model="gpt-4-1106-preview",
    temperature=0.0,
    max_tokens=50,
    response_format={"type": "json_object"},
)

history = ChatHistory().add_system_message("You are a helpfull chat assistant that helps the user answer questions.")
agent_pipeline = AgentPipeline(JSON_LLM,None,None,DATA_BASE_DESCRIPTORS)
clevr_decision_df = pd.read_csv("data/Decision_Evaluation/Databasedecision_Northwind.csv",sep=";")
clevr_decision_df["Chatbot Decision"] = ""
clevr_decision_df["Duration"] = None

In [4]:
prompt = agent_pipeline.generate_decision_prompt("In which movies did Keanu Reeves play?", history)

In [5]:
print(prompt)

Decide if you can answer the question only with the information of the chat history and if not which Database should be used to answer the question.
Important: You must answer in JSON format!
You have the follwing Databases available:
MovieDatabase: A neo4j database containing information about movies, actors and directors
CLEVR: A neo4j database containing information about underground train stations and underground train lines with their architecture and other addtitional informations
Northwind: A neo4j database containing an traditional retail-system with products, orders, customers, suppliers and employees

The question is:In which movies did Keanu Reeves play?
The current chat history is:
[{'role': 'system', 'content': 'You are a helpfull chat assistant that helps the user answer questions.'}]
Follow this example for the output:
{
  database: Literal["MovieDatabase", "CLEVR", "Northwind", "None"],
  can_answer_from_history: bool,
}


In [14]:
for index,row in tqdm(clevr_decision_df.iterrows(),desc="Generating Decisions"):
    question = row["Question"]
    start = time()
    decision = agent_pipeline.decide(question,history)
    end = time()
    duration = end - start
    row["Chatbot Decision"] = str(decision["database"])
    row["Duration"] = duration
    clevr_decision_df.to_csv("data/Decision_Evaluation/Databasedecision_Northwind_with_decision.csv",sep=";",index=False)

Generating Decisions: 44it [01:08,  1.56s/it]


In [15]:
df = pd.read_csv("data/Decision_Evaluation/Databasedecision_Northwind_with_decision.csv",sep=";")

In [16]:
avg_duration = df["Duration"].mean()

In [17]:
print(f"Average Duration: {avg_duration}")

Average Duration: 1.5561840371652083


In [18]:
(df["Expected Database"] == df["Chatbot Decision"]).sum() / len(df)

1.0

In [28]:
northwind_with_decisions = pd.read_csv("data/Decision_Evaluation/Databasedecision_Northwind_with_decision.csv",sep=";")
movies_with_decisions = pd.read_csv("data/Decision_Evaluation/Databasedecision_MovieDatabase_with_decision.csv",sep=";")
clevr_with_decisions = pd.read_csv("data/Decision_Evaluation/Databasedecision_CLEVR_with_decisions.csv",sep=";")
all_decisions_df = pd.concat([northwind_with_decisions,movies_with_decisions,clevr_with_decisions],ignore_index=True)

In [29]:
all_decisions_df.to_csv("data/Decision_Evaluation/All_Database_Decisions.csv",sep=";",index=False)

In [30]:
avg_duration = all_decisions_df["Duration"].mean()

In [31]:
print(f"Average Duration: {avg_duration}")

Average Duration: 1.9010669601724504


In [32]:
print(len(all_decisions_df))

282


In [33]:
print((all_decisions_df["Expected Database"] == all_decisions_df["Chatbot Decision"]).sum())

272


In [34]:
(all_decisions_df["Expected Database"] == all_decisions_df["Chatbot Decision"]).sum() / len(all_decisions_df)

0.9645390070921985

# Chat from History Evaluation

## 0 step reasoning

In [2]:
DATA_BASE_DESCRIPTORS = [
    DataBaseDescriptor(
        "MovieDatabase", "A neo4j database containing information about movies, actors and directors", None
    ),
        DataBaseDescriptor(
        "CLEVR", "A neo4j database containing information about underground train stations and underground train lines with their architecture and other addtitional informations", None
    ),
    DataBaseDescriptor(
        "Northwind", "A neo4j database containing an traditional retail-system with products, orders, customers, suppliers and employees", None
    )
]

JSON_LLM = OpenAILLM(
    api_key=os.environ.get("OPENAI_KEY"),
    model="gpt-4-1106-preview",
    temperature=0.0,
    max_tokens=50,
    response_format={"type": "json_object"},
)

history = ChatHistory().add_system_message("You are a helpfull chat assistant that helps the user answer questions.")
agent_pipeline = AgentPipeline(JSON_LLM,None,None,DATA_BASE_DESCRIPTORS)
clevr_decision_history_df = pd.read_csv("data/Decision_Evaluation/0_Step_History_Movie.csv",sep=";")
history_df_working = clevr_decision_history_df.copy()

In [3]:
history_df_working["Used History Chatbot Decision"] = ""
history_df_working["Duration"] = None

In [4]:
for index,row in tqdm(history_df_working.iterrows(),desc="Generating Decisions"):
    question = row["Question"]
    start = time()
    decision = agent_pipeline.decide(question,history)
    end = time()
    duration = end - start
    history_df_working.loc[index,"Used History Chatbot Decision"] = str(decision["can_answer_from_history"])
    history_df_working.loc[index,"Duration"] = duration
    history_df_working.to_csv("data/Decision_Evaluation/0_Step_History_Movie_with_decision.csv",sep=";",index=False)

Generating Decisions: 0it [00:00, ?it/s]

Generating Decisions: 114it [02:52,  1.51s/it]


## 1 step reasoning

In [7]:
DATA_BASE_DESCRIPTORS = [
    DataBaseDescriptor(
        "MovieDatabase", "A neo4j database containing information about movies, actors and directors", None
    ),
        DataBaseDescriptor(
        "CLEVR", "A neo4j database containing information about underground train stations and underground train lines with their architecture and other addtitional informations", None
    ),
    DataBaseDescriptor(
        "Northwind", "A neo4j database containing an traditional retail-system with products, orders, customers, suppliers and employees", None
    )
]

JSON_LLM = OpenAILLM(
    api_key=os.environ.get("OPENAI_KEY"),
    model="gpt-4-1106-preview",
    temperature=0.0,
    max_tokens=50,
    response_format={"type": "json_object"},
)

agent_pipeline = AgentPipeline(JSON_LLM,None,None,DATA_BASE_DESCRIPTORS)
clevr_decision_history_df = pd.read_csv("data/Decision_Evaluation/1_Step_History_Movie.csv",sep=";")
history_df_working = clevr_decision_history_df.copy()

In [8]:
history_df_working["Used History Chatbot Decision"] = ""
history_df_working["Duration"] = None

In [9]:
for index,row in tqdm(history_df_working.iterrows(),desc="Generating Decisions"):
    history = ChatHistory().add_system_message("You are a helpfull chat assistant that helps the user answer questions.")
    history.add_user_message(row["Question"])
    history.add_assistant_message(row["Answer Question"])
    question = row["Followup Question"]
    start = time()
    decision = agent_pipeline.decide(question,history)
    end = time()
    duration = end - start
    history_df_working.loc[index,"Used History Chatbot Decision"] = str(decision["can_answer_from_history"])
    history_df_working.loc[index,"Duration"] = duration
    history_df_working.to_csv("data/Decision_Evaluation/1_Step_History_Movie_with_decision.csv",sep=";",index=False)

Generating Decisions: 114it [02:39,  1.40s/it]


In [14]:
df = pd.read_csv("data/Decision_Evaluation/1_Step_History_Movie_with_decision.csv",sep=";")

In [15]:
avg_duration = df["Duration"].mean()

In [16]:
print(f"Average Duration: {avg_duration}")

Average Duration: 1.3932930502975196


In [17]:
accuracy = (df["Used History Expected"] == df["Used History Chatbot Decision"]).sum() / len(df)

In [18]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.8508771929824561


## 2 step reasoning

In [66]:
DATA_BASE_DESCRIPTORS = [
    DataBaseDescriptor(
        "MovieDatabase", "A neo4j database containing information about movies, actors and directors", None
    ),
        DataBaseDescriptor(
        "CLEVR", "A neo4j database containing information about underground train stations and underground train lines with their architecture and other addtitional informations", None
    ),
    DataBaseDescriptor(
        "Northwind", "A neo4j database containing an traditional retail-system with products, orders, customers, suppliers and employees", None
    )
]

JSON_LLM = OpenAILLM(
    api_key=os.environ.get("OPENAI_KEY"),
    model="gpt-4-1106-preview",
    temperature=0.0,
    max_tokens=50,
    response_format={"type": "json_object"},
)

agent_pipeline = AgentPipeline(JSON_LLM,None,None,DATA_BASE_DESCRIPTORS)
clevr_decision_history_df = pd.read_csv("data/Decision_Evaluation/2_Step_History_Movie.csv",sep=";")
history_df_working = clevr_decision_history_df.copy()

In [67]:
history_df_working["Used History Chatbot Decision"] = ""
history_df_working["Duration"] = None

In [68]:
for index,row in tqdm(history_df_working.iterrows(),desc="Generating Decisions"):
    history = ChatHistory().add_system_message("You are a helpfull chat assistant that helps the user answer questions.")
    history.add_user_message(row["Question 1"])
    history.add_assistant_message(row["Answer Question 1"])
    history.add_user_message(row["Question 2"])
    history.add_assistant_message(row["Answer Question 2"])
    question = row["Question 3"]
    start = time()
    decision = agent_pipeline.decide(question,history)
    end = time()
    duration = end - start
    history_df_working.loc[index,"Used History Chatbot Decision"] = str(decision["can_answer_from_history"])
    history_df_working.loc[index,"Duration"] = duration
    history_df_working.to_csv("data/Decision_Evaluation/2_Step_History_Movie_with_decision.csv",sep=";",index=False)

Generating Decisions: 114it [02:24,  1.27s/it]


In [69]:
df = pd.read_csv("data/Decision_Evaluation/2_Step_History_Movie_with_decision.csv",sep=";")

In [70]:
avg_duration = df["Duration"].mean()

In [71]:
print(f"Average Duration: {avg_duration}")

Average Duration: 1.2688024232262058


In [72]:
accuracy = (df["Used History Expected"] == df["Used History Chatbot Decision"]).sum() / len(df)

In [73]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.7543859649122807


## 3 Step reasoning

In [40]:
data = pd.read_csv("data/Decision_Evaluation/3_Step_History_Movie.csv",sep=";")

In [43]:
for index, row in data.iterrows():
    question_1 = row['Question 1'].replace('?', '').replace('.', '')
    question_2 = row['Question 2'].replace('?', '').replace('.', '')
    question_3 = row['Question 3'].replace('?', '').replace('.', '')
    question_4 = f"{question_1} and {question_2} and {question_3}?"
    data.loc[index, 'Question 4'] = question_4



In [44]:
data.to_csv("data/Decision_Evaluation/3_Step_History_Movie.csv",sep=";",index=False)

In [45]:
DATA_BASE_DESCRIPTORS = [
    DataBaseDescriptor(
        "MovieDatabase", "A neo4j database containing information about movies, actors and directors", None
    ),
        DataBaseDescriptor(
        "CLEVR", "A neo4j database containing information about underground train stations and underground train lines with their architecture and other addtitional informations", None
    ),
    DataBaseDescriptor(
        "Northwind", "A neo4j database containing an traditional retail-system with products, orders, customers, suppliers and employees", None
    )
]

JSON_LLM = OpenAILLM(
    api_key=os.environ.get("OPENAI_KEY"),
    model="gpt-4-1106-preview",
    temperature=0.0,
    max_tokens=50,
    response_format={"type": "json_object"},
)

agent_pipeline = AgentPipeline(JSON_LLM,None,None,DATA_BASE_DESCRIPTORS)
clevr_decision_history_df = pd.read_csv("data/Decision_Evaluation/3_Step_History_Movie.csv",sep=";")
history_df_working = clevr_decision_history_df.copy()

In [46]:
history_df_working["Used History Chatbot Decision"] = ""
history_df_working["Duration"] = None


In [47]:
for index,row in tqdm(history_df_working.iterrows(),desc="Generating Decisions"):
    history = ChatHistory().add_system_message("You are a helpfull chat assistant that helps the user answer questions.")
    history.add_user_message(row["Question 1"])
    history.add_assistant_message(row["Answer Question 1"])
    history.add_user_message(row["Question 2"])
    history.add_assistant_message(row["Answer Question 2"])
    history.add_user_message(row["Question 3"])
    history.add_assistant_message(row["Answer Question 3"])
    question = row["Question 4"]
    start = time()
    decision = agent_pipeline.decide(question,history)
    end = time()
    duration = end - start
    history_df_working.loc[index,"Used History Chatbot Decision"] = str(decision["can_answer_from_history"])
    history_df_working.loc[index,"Duration"] = duration
    history_df_working.to_csv("data/Decision_Evaluation/3_Step_History_Movie_with_decision.csv",sep=";",index=False)

Generating Decisions: 114it [02:38,  1.39s/it]


In [48]:
df = pd.read_csv("data/Decision_Evaluation/3_Step_History_Movie_with_decision.csv",sep=";")

In [49]:
avg_duration = df["Duration"].mean()

In [50]:
print(f"Average Duration: {avg_duration}")

Average Duration: 1.38554430844491


In [51]:
accuracy = (df["Used History Expected"] == df["Used History Chatbot Decision"]).sum() / len(df)

In [52]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.8596491228070176


# CQL Generation Evaluation

In [2]:
cql_evaluation_df = pd.read_json("questions.jsonl",lines=True)

In [3]:
random_lines = cql_evaluation_df.sample(n=500,replace=False,random_state=32)
random_lines_working = random_lines.copy()
random_lines_working.reset_index(drop=True,inplace=True)


In [12]:
CLEVR_DB_ADAPTER = Neo4jAdapter("bolt://localhost:7687", "", "").connect()

CHAT_LLM = OpenAILLM(api_key=os.environ.get("OPENAI_KEY"), model="gpt-4-1106-preview", temperature=0.2, max_tokens=200)
QUERY_LLM = OpenAILLM(api_key=os.environ.get("OPENAI_KEY"), model="gpt-4-1106-preview", temperature=0.0, max_tokens=100)


In [5]:
random_lines_working["cleaned_query"] = ""
random_lines_working["duration_query_generation"] = None
random_lines_working["db_result"] = ""
random_lines_working["syntax_error"] = False
random_lines_working["timeout_error"] = False
random_lines_working["duration_query_execution"] = None
random_lines_working["chat_generation_answer"] = ""
random_lines_working["duration_chat_answer"] = None

In [6]:
print(random_lines_working.columns)

Index(['question', 'cypher', 'group', 'answer', 'cleaned_query',
       'duration_query_generation', 'db_result', 'syntax_error',
       'duration_query_execution', 'chat_generation_answer',
       'duration_chat_answer'],
      dtype='object')


In [9]:
for index,row in tqdm(random_lines_working.iterrows(),desc="Generating Queries and Answers",total=len(random_lines_working)):
    history = ChatHistory().add_system_message("You are a helpfull chat assistant that helps the user answer questions.")
    question = row["question"]

    query_prompt = CLEVR_DB_ADAPTER.build_prompt(question)
    start_query_generation = time()
    cleaned_query = clean_generation(QUERY_LLM.generate(query_prompt))
    end_query_generation = time()
    duration_query_generation = end_query_generation - start_query_generation
    try:
        start_query_execution = time()
        db_result = CLEVR_DB_ADAPTER.execute(cleaned_query)
        end_query_execution = time()
        duration_query_execution = end_query_execution - start_query_execution

        start_chat_generation = time()
        chat_prompt = format_result_for_qa(question, db_result)
        history.add_user_message(chat_prompt)
        answer = CHAT_LLM.chat(history.format_for_model())
        end_chat_generation = time()
        duration_chat_answer = end_chat_generation-start_chat_generation
        db_result_str = json.dumps(db_result)
        syntax_error = False
        timeout_error = False

    except Exception as e:
        if isinstance(e, CypherSyntaxError):
            syntax_error = True
        elif isinstance(e, ClientError):
            timeout_error = True
        db_result = None
        db_result_str = None
        duration_query_execution = None
        duration_chat_answer = None
        answer = None
    

    random_lines_working.loc[index,"cleaned_query"] = cleaned_query
    random_lines_working.loc[index,"duration_query_generation"] = duration_query_generation
    random_lines_working.loc[index,"db_result"] = db_result_str
    random_lines_working.loc[index,"syntax_error"] = syntax_error
    random_lines_working.loc[index,"timeout_error"] = timeout_error
    random_lines_working.loc[index,"duration_query_execution"] = duration_query_execution
    random_lines_working.loc[index,"chat_generation_answer"] = answer
    random_lines_working.loc[index,"duration_chat_answer"] = duration_chat_answer

    random_lines_working.to_csv("data/Generation_Evaluation/Clevr_Generation_0_shot_secondtry.csv",sep=";",index=False)

Generating Queries and Answers: 300it [20:15,  4.05s/it]


In [21]:
results_df = pd.read_csv("data/Generation_Evaluation/Clevr_Generation_0_shot.csv",sep=";")

In [22]:
count = len(results_df[results_df["chat_generation_answer"] == "I don't know the answer."])
print(f"Count of 'I don't know the answer': {count}")

Count of 'I don't know the answer': 33


## Few Shot

In [8]:
zero_shot_df = pd.read_csv("data/Generation_Evaluation/Clevr_Generation_0_shot_with_timeout_error.csv",sep=";")

In [9]:
few_shot_df = pd.DataFrame(columns=zero_shot_df.columns)

In [10]:
print(few_shot_df.columns)

Index(['question', 'cypher', 'group', 'answer', 'cleaned_query',
       'duration_query_generation', 'db_result', 'syntax_error',
       'timeout_error', 'duration_query_execution', 'chat_generation_answer',
       'duration_chat_answer'],
      dtype='object')


In [13]:
FEW_SHOT_EXAMPLES_CLEVR = """
Question: How many stations are between Snoiarty St and Groitz Lane?
Cypher: MATCH path = shortestPath((start:STATION {name: 'Snoiarty St'})-[:EDGE*..50]-(end:STATION {name: 'Groitz Lane'}))
RETURN LENGTH(path) - 1 AS stationsBetween

Question: How many lines is the station Spriaords Palace on?
Cypher: MATCH (:STATION {name: "Spriaords Palace"})-[r:EDGE]-()
RETURN COUNT(DISTINCT r.line_id) AS NumberOfLines

Question: How many music styles does Orange Screic pass through?
Cypher: MATCH (s1:STATION)-[e:EDGE]->(s2:STATION) WHERE e.line_name = 'Orange Screic' UNWIND [s1.music, s2.music] AS musicStyle
RETURN COUNT(DISTINCT musicStyle)

Question: Can you get rail connections at Mclaewn Upon Thames?
Cypher: MATCH (s1:STATION {name: "Mclaewn Upon Thames"})-[:EDGE]->(s2:STATION)
RETURN s1.name AS Station, s2.name AS ConnectedStation, s1.has_rail AS HasRail

"""

In [14]:
CLEVR_DB_ADAPTER = Neo4jAdapter("bolt://localhost:7687", "", "",FEW_SHOT_EXAMPLES_CLEVR).connect()

CHAT_LLM = OpenAILLM(api_key=os.environ.get("OPENAI_KEY"), model="gpt-4-1106-preview", temperature=0.2, max_tokens=200)
QUERY_LLM = OpenAILLM(api_key=os.environ.get("OPENAI_KEY"), model="gpt-4-1106-preview", temperature=0.0, max_tokens=100)

In [15]:
for index,row in tqdm(zero_shot_df.iterrows(),desc="Generating Queries and Answers",total=len(zero_shot_df)):
    history = ChatHistory().add_system_message("You are a helpfull chat assistant that helps the user answer questions.")
    question = row["question"]

    query_prompt = CLEVR_DB_ADAPTER.build_prompt(question)
    start_query_generation = time()
    cleaned_query = clean_generation(QUERY_LLM.generate(query_prompt))
    end_query_generation = time()
    duration_query_generation = end_query_generation - start_query_generation
    try:
        start_query_execution = time()
        db_result = CLEVR_DB_ADAPTER.execute(cleaned_query)
        end_query_execution = time()
        duration_query_execution = end_query_execution - start_query_execution

        start_chat_generation = time()
        chat_prompt = format_result_for_qa(question, db_result)
        history.add_user_message(chat_prompt)
        answer = CHAT_LLM.chat(history.format_for_model())
        end_chat_generation = time()
        duration_chat_answer = end_chat_generation-start_chat_generation
        db_result_str = json.dumps(db_result)
        syntax_error = False
        timeout_error = False

    except Exception as e:
        if isinstance(e, CypherSyntaxError):
            syntax_error = True
        elif isinstance(e, ClientError):
            timeout_error = True
        db_result = None
        db_result_str = None
        duration_query_execution = None
        duration_chat_answer = None
        answer = None
    
    few_shot_df.loc[index,"question"] = question
    few_shot_df.loc[index,"cypher"] = row["cypher"]
    few_shot_df.loc[index,"group"] = row["group"]
    few_shot_df.loc[index,"answer"] = row["answer"]
    few_shot_df.loc[index,"cleaned_query"] = cleaned_query
    few_shot_df.loc[index,"duration_query_generation"] = duration_query_generation
    few_shot_df.loc[index,"db_result"] = db_result_str
    few_shot_df.loc[index,"syntax_error"] = syntax_error
    few_shot_df.loc[index,"timeout_error"] = timeout_error
    few_shot_df.loc[index,"duration_query_execution"] = duration_query_execution
    few_shot_df.loc[index,"chat_generation_answer"] = answer
    few_shot_df.loc[index,"duration_chat_answer"] = duration_chat_answer

    few_shot_df.to_csv("data/Generation_Evaluation/Clevr_Generation_few_shot_with_timeout_error.csv",sep=";",index=False)

Generating Queries and Answers: 100%|██████████| 500/500 [36:20<00:00,  4.36s/it]


In [16]:
result = pd.read_csv("data/Generation_Evaluation/Clevr_Generation_few_shot_with_timeout_error.csv",sep=";")


In [17]:
syntax_errors = result["syntax_error"].sum()
timeout_errors = result["timeout_error"].sum()

print(f"Number of syntax errors: {syntax_errors}")
print(f"Number of timeout errors: {timeout_errors}")


Number of syntax errors: 0
Number of timeout errors: 0


In [18]:
empty_result_count = result['db_result'].value_counts()['[]']
print(f"Count of empty db results: {empty_result_count}")


Count of empty db results: 27


In [20]:
count = len(result[result["chat_generation_answer"] == "I don't know the answer."])
print(f"Count of 'I don't know the answer': {count}")


Count of 'I don't know the answer': 32


# Analysis of results

## 0-Step

In [2]:
zero_step_history_celvr_df = pd.read_csv("data/Decision_Evaluation/0_Step_History_Clevr_with_decision.csv",sep=";")
zero_step_history_movie_df = pd.read_csv("data/Decision_Evaluation/0_Step_History_Movie_with_decision.csv",sep=";")

In [1]:
print(len(zero_step_history_movie_df))

NameError: name 'zero_step_history_movie_df' is not defined

In [3]:
all_zero_step_history_df = pd.concat([zero_step_history_celvr_df,zero_step_history_movie_df],ignore_index=True)

In [4]:
print(len(all_zero_step_history_df))

214


In [8]:
print((all_zero_step_history_df["Used History Expected"] == all_zero_step_history_df["Used History Chatbot Decision"]).sum() / len(all_zero_step_history_df))

1.0


In [9]:
print(all_zero_step_history_df["Duration"].mean())

1.5628224232486476


## 1-Step

In [10]:
one_step_history_celvr_df = pd.read_csv("data/Decision_Evaluation/1_Step_History_Clevr_with_decision.csv",sep=";")
one_step_history_movie_df = pd.read_csv("data/Decision_Evaluation/1_Step_History_Movie_with_decision.csv",sep=";")

In [12]:
print(len(one_step_history_celvr_df))

100


In [8]:
all_one_step_history_df = pd.read_csv("data/Decision_Evaluation/All_One_Step_History_Decisions.csv",sep=";")

In [5]:
print(len(all_one_step_history_df))

214


In [15]:
print((all_one_step_history_df["Used History Expected"] == all_one_step_history_df["Used History Chatbot Decision"]).sum() / len(all_one_step_history_df))

0.822429906542056


In [16]:
print(all_one_step_history_df["Duration"].mean())

1.5194765619028394


## 2-Step

In [17]:
two_step_history_celvr_df = pd.read_csv("data/Decision_Evaluation/2_Step_History_Clevr_with_decision.csv",sep=";")
two_step_history_movie_df = pd.read_csv("data/Decision_Evaluation/2_Step_History_Movie_with_decision.csv",sep=";")

In [19]:
print(len(two_step_history_movie_df))

114


In [9]:
all_two_step_history_df = pd.read_csv("data/Decision_Evaluation/All_Two_Step_History_with_Decisions.csv",sep=";")

In [10]:
print(len(all_two_step_history_df))

214


In [22]:
print((all_two_step_history_df["Used History Expected"] == all_two_step_history_df["Used History Chatbot Decision"]).sum() / len(all_two_step_history_df))

0.794392523364486


In [23]:
print(all_two_step_history_df["Duration"].mean())

1.5202398756954159


## 3-Step

In [24]:
three_step_history_celvr_df = pd.read_csv("data/Decision_Evaluation/3_Step_History_Clevr_with_decision.csv",sep=";")
three_step_history_movie_df = pd.read_csv("data/Decision_Evaluation/3_Step_History_Movie_with_decision.csv",sep=";")

In [26]:
print(len(three_step_history_movie_df))

114


In [11]:
all_three_step_history_df = pd.read_csv("data/Decision_Evaluation/All_Three_Step_History_with_Decisions.csv",sep=";")

In [12]:
print(len(all_three_step_history_df))

214


In [29]:
print((all_three_step_history_df["Used History Expected"] == all_three_step_history_df["Used History Chatbot Decision"]).sum() / len(all_three_step_history_df))

0.8738317757009346


In [30]:
print(all_three_step_history_df["Duration"].mean())

1.6882413982230926


## DB Decision

In [31]:
all_db_decisions_df = pd.read_csv("data/Decision_Evaluation/All_Database_Decisions.csv",sep=";")

In [32]:
print(len(all_db_decisions_df))

282


In [33]:
print((all_db_decisions_df["Expected Database"] == all_db_decisions_df["Chatbot Decision"]).sum() / len(all_db_decisions_df))

0.9645390070921985


In [34]:
print(all_db_decisions_df["Duration"].mean())

1.9010669601724504


## 0-shot

In [14]:
zero_shot_df = pd.read_csv("data/Generation_Evaluation/Clevr_Generation_0_shot_with_timeout_error_with_evaluation.csv",sep=";")

In [15]:
syntax_errors = zero_shot_df["syntax_error"].sum()
timeout_errors = zero_shot_df["timeout_error"].sum()
print(f"Number of syntax errors: {syntax_errors}")
print(f"Number of timeout errors: {timeout_errors}")

Number of syntax errors: 48
Number of timeout errors: 20


In [16]:
duration_query_generation = zero_shot_df["duration_query_generation"].mean()
duration_query_execution = zero_shot_df["duration_query_execution"].mean()
duration_chat_answer = zero_shot_df["duration_chat_answer"].mean()

print(f"Average duration query generation: {duration_query_generation}")
print(f"Average duration query execution: {duration_query_execution}")
print(f"Average duration chat answer: {duration_chat_answer}")

Average duration query generation: 3.290248386859894
Average duration query execution: 0.015288315350384234
Average duration chat answer: 1.7156024537894123


In [17]:
execution_accuracy = (zero_shot_df["Human Evaluation"] == "Correct").sum() / len(zero_shot_df)

In [18]:
print(f"Execution Accuracy: {execution_accuracy}")

Execution Accuracy: 0.61


## Few-Shot

In [25]:
few_shot_df = pd.read_csv("data/Generation_Evaluation/Clevr_Generation_few_shot_with_timeout_error_with_evaluation.csv",sep=";")

In [26]:
syntax_errors = few_shot_df["syntax_error"].sum()
timeout_errors = few_shot_df["timeout_error"].sum()
print(f"Number of syntax errors: {syntax_errors}")
print(f"Number of timeout errors: {timeout_errors}")

Number of syntax errors: 0
Number of timeout errors: 0


In [27]:
duration_query_generation = few_shot_df["duration_query_generation"].mean()
duration_query_execution = few_shot_df["duration_query_execution"].mean()
duration_chat_answer = few_shot_df["duration_chat_answer"].mean()

print(f"Average duration query generation: {duration_query_generation}")
print(f"Average duration query execution: {duration_query_execution}")
print(f"Average duration chat answer: {duration_chat_answer}")

Average duration query generation: 3.008542133808136
Average duration query execution: 0.009419580459594677
Average duration chat answer: 1.3377295742034911


In [28]:
execution_accuracy = (few_shot_df["Human Evaluation"] == "Correct").sum() / len(few_shot_df)

In [29]:
print(f"Execution Accuracy: {execution_accuracy}")

Execution Accuracy: 0.928


DB Decision

In [1]:
import pandas as pd

# Read the file
df = pd.read_csv("data/Decision_Evaluation/All_Database_Decisions.csv", sep=";")

filtered_df_movie = df[df["Expected Database"] == "MovieDatabase"]
filtered_df_clevr = df[df["Expected Database"] == "CLEVR"]
filtered_df_northwind = df[df["Expected Database"] == "None"]
filtered_df_none = df[df["Expected Database"] == "Northwind"]


In [2]:
accuracy_movie = (filtered_df_movie["Expected Database"] == filtered_df_movie["Chatbot Decision"]).mean()
accuracy_clevr = (filtered_df_clevr["Expected Database"] == filtered_df_clevr["Chatbot Decision"]).mean()
accuracy_northwind = (filtered_df_northwind["Expected Database"] == filtered_df_northwind["Chatbot Decision"]).mean()
accuracy_none = (filtered_df_none["Expected Database"] == filtered_df_none["Chatbot Decision"]).mean()

print(f"Accuracy for MovieDatabase: {accuracy_movie}")
print(f"Accuracy for CLEVR: {accuracy_clevr}")
print(f"Accuracy for Northwind: {accuracy_northwind}")
print(f"Accuracy for None: {accuracy_none}")


Accuracy for MovieDatabase: 1.0
Accuracy for CLEVR: 0.9
Accuracy for Northwind: 1.0
Accuracy for None: 1.0


In [7]:

# Calculate the mean time for decision per database
mean_time_movie = filtered_df_movie["Duration"].mean()
mean_time_clevr = filtered_df_clevr["Duration"].mean()
mean_time_northwind = filtered_df_northwind["Duration"].mean()
mean_time_none = filtered_df_none["Duration"].mean()

# Print the results
print(f"Mean time for decision in MovieDatabase: {mean_time_movie}")
print(f"Mean time for decision in CLEVR: {mean_time_clevr}")
print(f"Mean time for decision in Northwind: {mean_time_northwind}")
print(f"Mean time for decision in None: {mean_time_none}")


Mean time for decision in MovieDatabase: 1.7899449598991264
Mean time for decision in CLEVR: 2.206049556732178
Mean time for decision in Northwind: 1.7431732575098673
Mean time for decision in None: 1.529036003000596


In [3]:
# caculate the weigted accuracy for all databases
weighted_accuracy = (accuracy_movie * len(filtered_df_movie) + accuracy_clevr * len(filtered_df_clevr) + accuracy_northwind * len(filtered_df_northwind)+ accuracy_none * len(filtered_df_none))/len(df)

In [4]:
print(f"Weighted Accuracy: {weighted_accuracy}")

Weighted Accuracy: 0.9645390070921985


In [5]:
print(len(filtered_df_movie))
print(len(filtered_df_clevr))
print(len(filtered_df_northwind))
print(len(filtered_df_none))

118
100
30
34
